# 训练instance segmentation模型

In [ ]:
import wandb
wandb.login(key="957096cc564005d5332d45e2da6a75838e1cc9ac")

In [ ]:
from lark import logger
from ultralytics import YOLO

# ===== project name =====
PROJECT_NAME = "swd_model_v1"
runPath = "/workspace/_ty/03_code/05_train_model/yolo/yaml/"
# ========================

# ==== 你需要手动填写的列表 ====
yamlFileNames = [  # 数据集 yaml 文件名（不要写后缀 .yaml）
    "data_split_0.4_0.3_0.3",
    # "data_split_0.5_0.2_0.3",
    "data_split_0.5_0.3_0.2",
    "data_split_0.6_0.2_0.2",
    "data_split_0.8_0.2_0",
    "data_split_0.7_0.2_0.1",
]
batchSizes = [
    # 4, 
    8, 16, 32, 
    # 64,
]
models = [
    # "yolo11n-seg.pt",
    # "yolo11s-seg.pt",
    "yolo11n.pt",
    "yolo11s.pt",
    # "yolo11m-seg.pt",
    "yolo11m.pt",
    # "yolo11l-seg.pt", "yolo11x-seg.pt",
    # "yolo11l.pt", "yolo11x.pt",
]
# models = [
#     "yolo11n-seg.pt",
#     "/workspace/_ty/models/runs_yolov11/output_16mp/good/yolo11m-seg.pt---data_split_0.6_0.2_0.2_8-----0.909/weights/best.pt",
#     "/workspace/_ty/models/runs_yolov11/output_16mp/good/yolo11n-seg.pt---data_split_0.6_0.2_0.2_4---0.913/weights/best.pt",
#     "/workspace/_ty/models/runs_yolov11/output_16mp/good/yolo11n.pt---data_split_0.6_0.1_0.3_4----0.906/weights/best.pt",
# ]
# =================================

# 循环遍历
for yamlFileName in yamlFileNames:
    yamlPath = runPath + yamlFileName + ".yaml"
    for modelFile in models:
        for batch in batchSizes:
            print(f"\n🚀 Training model={modelFile}, dataset={yamlFileName}, batch={batch}")

            model = YOLO(modelFile)

            try:
                model.train(
                    data=yamlPath,
                    epochs=10,
                    imgsz=640,
                    batch=batch,
                    device=0,
                    workers=4,
                    project=f"output/{PROJECT_NAME}",   # 输出目录
                    name=f"{modelFile}_{yamlFileName}_{batch}",
                )

                # 测试集验证
                model.val(
                    data=yamlPath,
                    split="test",
                    name=f"{modelFile}_{yamlFileName}_{batch}_test",
                )
            except RuntimeError as e:
                if "CUDA out of memory" in str(e):
                    print(f"⚠️  跳过: model={modelFile}, yaml={yamlFileName}, batch={batch} —— 显存不足")
                    continue
                else:
                    raise  # 不是 OOM 的错误则继续抛出

# 单个例子

In [ ]:
# runPath = "/workspace/models/runs_yolov11/yaml/"
# yamlFileName = "data_split_v1_0.5_0.2_0.3"
# yamlPath = runPath + yamlFileName + ".yaml"

In [ ]:
# from ultralytics import YOLO
#
# runPath = "/workspace/models/runs_yolov11/"
# yamlFileName = "data_split_v1_0.5_0.2_0.3"
# yamlPath = runPath + yamlFileName + ".yaml"
#
# model = YOLO("yolo11n-seg.pt")  # load a pretrained model (recommended for training)
#
# # Train the model
# model.train(
#     data=yamlPath,  # 数据路径（文件夹结构符合分类任务格式）
#     epochs=1000,                   # 训练轮数
#     imgsz=640,                   # 输入图像大小
#     batch=64,                    # 批大小
#     device=0,                    # GPU ID
#     workers=4,                    # DataLoader 线程数
#     project="output/runs_yolo11n-seg_batch64",
#     name=yamlFileName
# )
# model.val(data=yamlPath, split="test", name=yamlFileName+"_test")

In [ ]:
# from ultralytics import YOLO
# # model = YOLO("/workspace/models/runs_yolov11/output/yolo11m-seg.pt/data_split_0.6_0.2_0.2_8-----0.909/weights/best.pt")  # load a pretrained model (recommended for training)
# model = YOLO("/workspace/models/runs_yolov11/output/good/yolo11n-seg.pt---data_split_0.6_0.2_0.2_4---0.913/weights/best.pt")  # load a pretrained model (recommended for training)

# model.val(data="/workspace/models/runs_yolov11/yaml/data_split_0.6_0.2_0.2.yaml", split="test")